In [1]:
pip install --upgrade mxnet matplotlib sagemaker Pillow

Note: you may need to restart the kernel to use updated packages.


In [2]:
import numpy as np
from numpy import asarray
import sagemaker.amazon.common as smac
import io
import cv2
import glob

# Open the image form working directory
def read_image(cv2_image_name):
    image_height_rs = 324
    image_width_rs = 486

    cv2_image=cv2.imread(cv2_image_name)
    # resize image
    cv2_resized_image = cv2.resize(cv2_image, (image_height_rs, image_width_rs))
    # reshape image
    cv2_reshaped_image = np.reshape(cv2_resized_image, cv2_resized_image.shape[0]*cv2_resized_image.shape[1]*cv2_resized_image.shape[2])
    return cv2_reshaped_image

# Load images to train the model
file_name_list = []
train_set_vectors = []
train_set_labels_array = []
for filename in glob.glob('images/*'):
    file_name_list.append(filename)
    cv2_image=read_image(filename)
    label_value=0 if ("_invalid" in filename) else 1
    train_set_vectors.append(cv2_image)    
    #Add labels (0 if image name includes _invalid keyword, otherwise 1)
    train_set_labels_array.append(np.int_(label_value))

vectors = np.array([t.tolist() for t in train_set_vectors]).astype("float32")
train_set_labels = np.array(train_set_labels_array)
labels=train_set_labels.astype("float32")

buf = io.BytesIO()
smac.write_numpy_to_dense_tensor(buf, vectors, labels)
buf.seek(0)

0

In [3]:
import boto3
import os
import sagemaker

sess = sagemaker.Session()
key = "scenechanges-data"
prefix = "sagemaker/aws-iot-blog-scenechanges"
bucket = sess.default_bucket()

boto3.resource("s3").Bucket(bucket).Object(os.path.join(prefix, "train", key)).upload_fileobj(buf)
s3_train_data = f"s3://{bucket}/{prefix}/train/{key}"
print(f"uploaded training data location: {s3_train_data}")

output_location = f"s3://{bucket}/{prefix}/output"
print(f"training artifacts will be uploaded to: {output_location}")

uploaded training data location: s3://sagemaker-us-east-1-627316285343/sagemaker/aws-iot-blog-scenechanges/train/scenechanges-data
training artifacts will be uploaded to: s3://sagemaker-us-east-1-627316285343/sagemaker/aws-iot-blog-scenechanges/output


In [4]:
from sagemaker.amazon.amazon_estimator import get_image_uri
from sagemaker import get_execution_role

container = sagemaker.image_uris.retrieve("linear-learner", boto3.Session().region_name)
role = get_execution_role()

In [5]:
import boto3
import sagemaker

sess = sagemaker.Session()

linear = sagemaker.estimator.Estimator(
    container,
    role,
    instance_count=1,
    instance_type="ml.c5n.2xlarge",
    output_path=output_location,
    sagemaker_session=sess,
)

linear.set_hyperparameters(feature_dim="auto", predictor_type="binary_classifier",mini_batch_size=1)

result = linear.fit({"train": s3_train_data})
print(result)

2021-06-07 14:09:07 Starting - Starting the training job...
2021-06-07 14:09:31 Starting - Launching requested ML instancesProfilerReport-1623074947: InProgress
.........
2021-06-07 14:10:51 Starting - Preparing the instances for training......
2021-06-07 14:12:09 Downloading - Downloading input data
2021-06-07 14:12:09 Training - Downloading the training image...
2021-06-07 14:12:32 Training - Training image download completed. Training in progress.Docker entrypoint called with argument(s): train
Running default environment configuration script
[06/07/2021 14:12:28 INFO 140185238931264] Reading default configuration from /opt/amazon/lib/python3.7/site-packages/algorithm/resources/default-input.json: {'mini_batch_size': '1000', 'epochs': '15', 'feature_dim': 'auto', 'use_bias': 'true', 'binary_classifier_model_selection_criteria': 'accuracy', 'f_beta': '1.0', 'target_recall': '0.8', 'target_precision': '0.8', 'num_models': 'auto', 'num_calibration_samples': '10000000', 'init_method': '

In [6]:
linear_predictor = linear.deploy(initial_instance_count=1, instance_type="ml.m4.xlarge")

-----------------!